<a href="https://colab.research.google.com/github/agemagician/ProtTrans/blob/master/Embedding/TensorFlow/Advanced/ProtBert-BFD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3> Extracting protein sequences' features using ProtBert-BFD pretrained-model <h3>

<b>1. Load necessry libraries including huggingface transformers<b>

In [1]:
!pip install -q transformers

     |████████████████████████████████| 1.1MB 4.9MB/s 
     |████████████████████████████████| 1.1MB 23.0MB/s 
     |████████████████████████████████| 890kB 45.9MB/s 
     |████████████████████████████████| 3.0MB 14.2MB/s 


In [2]:
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer,BertConfig
import re
import numpy as np

<b>2. Load the vocabulary and ProtBert-BFD Model</b>

In [3]:
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert_bfd", do_lower_case=False )

In [4]:
model = TFBertModel.from_pretrained("Rostlab/prot_bert_bfd")

All model checkpoint weights were used when initializing TFBertModel.

All the weights of TFBertModel were initialized from the model checkpoint at Rostlab/prot_bert_bfd.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


<b>3. Create or load sequences and map rarely occured amino acids (U,Z,O,B) to (X)<b>

In [5]:
sequences_Example = ["A E T C Z A O","S K T Z P"]

In [6]:
sequences_Example = [re.sub(r"[UZOB]", "X", sequence) for sequence in sequences_Example]

<b>4. Tokenize, encode sequences and load it into the GPU if possibile<b>

In [7]:
ids = tokenizer.batch_encode_plus(sequences_Example, add_special_tokens=True, pad_to_max_length=True, return_tensors="tf")

/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:1773: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [8]:
input_ids = ids['input_ids']
attention_mask = ids['attention_mask']

<b>5. Extracting sequences' features and load it into the CPU if needed<b>

In [9]:
embedding = model(input_ids)[0]

In [10]:
embedding = np.asarray(embedding)

In [11]:
attention_mask = np.asarray(attention_mask)

<b>6. Remove padding ([PAD]) and special tokens ([CLS],[SEP]) that is added by ProtBert-BFD model<b>

In [12]:
features = [] 
for seq_num in range(len(embedding)):
    seq_len = (attention_mask[seq_num] == 1).sum()
    seq_emd = embedding[seq_num][1:seq_len-1]
    features.append(seq_emd)

In [13]:
print(features)

[array([[-0.6270308 ,  0.34281123, -0.8005184 , ...,  0.91371715,
         0.31192842, -0.69043314],
       [-0.54374063, -0.06574883, -0.8056429 , ...,  0.54859066,
         0.57313085, -0.5818112 ],
       [-0.49946   , -0.09692474, -0.58833486, ...,  0.8908874 ,
        -0.29026067, -0.27423018],
       ...,
       [-0.47744069,  0.04738433, -1.1085062 , ...,  0.93225986,
         0.041368  , -0.19019172],
       [-0.45420837, -0.09183727, -0.63870645, ...,  1.0848563 ,
         0.3077175 , -0.33701992],
       [-0.39709502,  0.326098  , -1.0061178 , ...,  1.1735164 ,
        -0.202245  , -0.13552478]], dtype=float32), array([[-0.2248441 ,  0.29396212, -1.3402402 , ...,  0.60739344,
         0.691832  , -0.37161052],
       [-0.40438125,  0.09351258, -1.0968096 , ...,  0.48171398,
         0.40898085, -0.47887608],
       [-0.5430368 , -0.3650298 , -0.7755365 , ...,  0.66056466,
        -0.05191286, -0.4356588 ],
       [-0.21376015,  0.2360549 , -1.5505239 , ...,  0.754462  ,
     